In [ ]:
%load_ext ext.stackql

In [ ]:
region = 'us-east-1'

In [ ]:
%%stackql
SELECT 
split_part(instanceState, '\n', 3) as instanceState, 
count(*) as num_instances
FROM aws.ec2.instances 
WHERE region = '$region' 
GROUP BY split_part(instanceState, '\n', 3)

In [ ]:
_.plot(kind='bar', title='Instances by State', x='instanceState', y='num_instances')

In [ ]:
%%stackql
SELECT instanceType, COUNT(*) as num_instances 
FROM aws.ec2.instances
WHERE region = '$region'
GROUP BY instanceType

In [ ]:
_.plot(kind='pie', y='num_instances', labels=_['instanceType'], title='Instances by Type', autopct='%1.1f%%')

In [ ]:
# get multiple regions asynchronously
import nest_asyncio, json
nest_asyncio.apply()
from pystackql import StackQL
import pandas as pd

stackql = StackQL()

regions = ["ap-southeast-2", "us-east-1"]

queries = [
    f"""
    SELECT '{region}' as region, instanceType, COUNT(*) as num_instances
    FROM aws.ec2.instances
    WHERE region = '{region}'
    GROUP BY instanceType
    """
    for region in regions
]

res = stackql.executeQueriesAsync(queries)
instances_df = pd.read_json(json.dumps(res))